In [ ]:
from pathlib import Path
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from scipy.cluster.hierarchy import linkage, dendrogram
import seaborn as sns

from sklearn.cluster import (
    KMeans,
    MiniBatchKMeans,
    AgglomerativeClustering,
    DBSCAN,
    MeanShift,
    estimate_bandwidth,
)
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.utils import resample, random
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV
import warnings

warnings.filterwarnings("ignore")
import pathlib

# ***********notes*******************
# NOTE: Alert Type Meanings:
# HHA= high high alert
# LLA = Low low alert
# AAF = Area Alert - Fast Response
# AAS: areal alert - slow response
# FRZ: Frozen Data
# FRQ: Anomaly Frequency
# OSC: Oscillations
# WOR: Watch Override


In [ ]:
# GENERAL PLOT FUNCTIONS:
def plt_heat(df_toplt, plt_name="1", fig_hight = 0, fig_width = 0, fntsze=12):
    # Correlation between columns
    if fig_hight ==0:
        fig_hight = len(df_toplt.columns)
    if fig_width ==0:
        fig_width = len(df_toplt.columns)

    figsize = (fig_width, fig_hight)
    plt.figure(figsize=figsize)
    sns.heatmap(
        data=df_toplt.corr(),
        annot=True,
        linewidths=0.05,
        cmap="coolwarm",
        square=True,
        fmt=".2f",
        annot_kws={"fontsize":fntsze}
    ).tick_params(labelsize=fntsze+2)

    
    # plt.figure.yaxis('', size=36)
    # plt.figure.set_xlabel('', size=36)
    plt.title("CORRELATION MATRIX", fontsize=fntsze, color="dimgrey", fontweight="bold")
    plt.savefig(f"trials/v_{version}/data_images/heatmap_{plt_name}.png")
    plt.show()


def plt_boxplot(df_toplt, plt_name="1", fig_hight = 0, fig_width = 0, fntsze=12):
    
    if fig_hight ==0:
        fig_hight = len(df_toplt.columns)
    if fig_width ==0:
        fig_width = len(df_toplt.columns)+10

    figsize = (fig_width, fig_hight)

    # Listing numeric columns
    v_n = list(df_toplt.select_dtypes(include=["float64", "int64"]).columns)

    # Plot
    plt.figure(figsize=figsize)
    sns.set(font_scale=1)
    x = 1
    for column in v_n:
        plt.subplot(((df_toplt.shape[1] // 3) + 1), 3, x)
        sns.boxplot(df_toplt[column]).tick_params(labelsize=fntsze)
        plt.title(
            "BoxPlot {}".format(column), fontsize=fntsze, color="dimgrey", fontweight="bold"
        )
        x += 1
    plt.savefig(f"trials/v_{version}/data_images/box_{plt_name}.png")


def plt_distplot(df_toplt, plt_name="1", fig_hight = 0, fig_width = 0, fntsze=12):

    if fig_hight ==0:
        fig_hight = len(df_toplt.columns)
    if fig_width ==0:
        fig_width = len(df_toplt.columns)+10

    figsize = (fig_width, fig_hight)

    v_n = list(df_toplt.select_dtypes(include=["float64", "int64"]).columns)
    plt.figure(figsize=figsize)
    x = 1
    sns.set(font_scale=1)
    
    for column in v_n:
        
        plt.subplot(((df_toplt.shape[1] // 3) + 1), 3, x)
        sns.distplot(df_toplt[column], kde=True, norm_hist=True, bins=100).tick_params(labelsize=fntsze)
        plt.title(
            "Distribution {}".format(column),
            fontsize=fntsze,
            color="dimgrey",
            fontweight="bold",
            y=1,
        )
        x += 1

    plt.savefig(f"trials/v_{version}/data_images/histogram_{plt_name}.png")


def plt_histo(df_toplt, plt_name="1", fig_hight = 0, fig_width = 0, fntsze=12, bns=60):

    if fig_hight ==0:
        fig_hight = len(df_toplt.columns)+10
    if fig_width ==0:
        fig_width = len(df_toplt.columns)+15

    figsize = (fig_width, fig_hight)

    v_n = list(df_toplt.select_dtypes(include=["float64", "int64"]).columns)
    plt.figure(figsize=figsize)
    x = 1
    for column in v_n:
        plt.subplot(((df_toplt.shape[1] // 3) + 1), 3, x)
        plt.hist(df_toplt[column], bins=bns, linewidth=0.5, edgecolor="white")
        plt.xticks(fontsize=fntsze)
        plt.title(
            "Distribution {}".format(column),
            fontsize=fntsze+2,
            color="dimgrey",
            fontweight="bold",
            y=1,
        )
        x += 1
    

def variance_bars_plt(plt_name="1"):
    # Instantiate the Percentage of Variance Explained
    pve = pca.explained_variance_ratio_
    # Instantiate the Cumulative Percentage of the Variance
    pcve = np.cumsum(pve)

    plt.bar(
        range(0, len(pve)),
        pve,
        alpha=0.5,
        align="center",
        label="Percentage of Individually Explained Variance",
        color="blue",
    )
    plt.step(
        range(0, len(pcve)),
        pcve,
        where="mid",
        label="Cumulative Percentage of Explained Variance",
        color="red",
    )
    plt.xticks(range(0, pca.n_features_in_))
    plt.ylabel("Percentage of Explained Variance")
    plt.xlabel("Components")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(f"trials/v_{version}/data_images/variance_bars_{plt_name}.png")
    plt.show()


def percent_variance_plt_ratio(
    plots, yaxis_title, xaxis_title, dashed_pos, plt_name="1"
):
    rows = (len(plots) // 2) + 1
    plt.figure(figsize=(20, 20))
    for i in range(len(plots)):
        plt.subplot(rows, 2, i + 1)
        plt.style.use("ggplot")

        if dashed_pos[i] > 0.0:
            plt.axhline(y=dashed_pos[i], color="black", linestyle="--")

        plt.plot(plots[i], marker="o")
        plt.xticks(range(0, pca.n_features_in_))
        plt.xlabel(xaxis_title[i])
        plt.ylabel(yaxis_title[i])
        plt.title("Scree Plot")

    plt.subplots_adjust(wspace=0.4)
    plt.savefig(f"trials/v_{version}/data_images/variance_ratios_{plt_name}.png")
    plt.show()


def plt_kelbow_metric(data, model_touse, k_range, metric_touse, visualizer_touse):
    rows = (len(metric_touse) // 2) + 1

    plt.figure(figsize=(20, 15))
    plt.rcParams["xtick.labelsize"] = 8
    plt.rcParams["ytick.labelsize"] = 8

    for i in range(len(metric_touse)):
        plt.subplot(rows, 2, i + 1)
        if visualizer_touse == "kmeans":
            visualizer = KElbowVisualizer(
                model_touse, k=k_range, metric=metric_touse[i], timings=False
            )
        elif visualizer_touse == "AgglomerativeClustering":
            visualizer = KElbowVisualizer(
                AgglomerativeClustering(), metric=metric_touse[i], timings=False
            )
        visualizer.fit(data)
        visualizer.finalize()

    plt.subplots_adjust(wspace=0.4)
    plt.savefig(f"trials/v_{version}/scatter_images/kelbow_{visualizer_touse}.png")
    plt.show()


def plot_kmeans_plotly(xycol, kmeans, ttle):
    x_col = col_index_name[xycol[0]]
    y_col = col_index_name[xycol[1]]

    fig = px.scatter(
        x=features_df.iloc[:, x_col[0]],
        y=features_df.iloc[:, y_col[0]],
        color=kmeans.labels_,
        title=ttle,
    ).update_layout(xaxis_title=x_col[1], yaxis_title=y_col[1])
    return fig


def plot_general(df, xycol, plt_name='', figsiz=(20, 15), fntsze=10):
    col_index_name = [(df.columns.get_loc(c), c) for c in df]
    rows = (len(xycol) // 2) + 1
    plt.figure(figsize=figsiz)
    plt.rcParams["xtick.labelsize"] = fntsze
    plt.rcParams["ytick.labelsize"] = fntsze

    for i in range(len(xycol)):
        # i=0
        x_col = col_index_name[xycol[i][0]]
        y_col = col_index_name[xycol[i][1]]

        plt.subplot(rows, 2, i + 1)
        sns.scatterplot(data=df, x=x_col[1], y=y_col[1], hue="cluster", palette="Set2")
        plt.xticks(fontsize=fntsze)
        plt.title(f"{x_col[1]} vs {y_col[1]}", fontsize=fntsze+2, color="dimgrey", fontweight="bold")
        plt.xlabel(x_col[1], color="dimgrey", labelpad=5, fontweight="bold", fontsize=fntsze)
        plt.ylabel(y_col[1], color="dimgrey", fontweight="bold", fontsize=fntsze)

    plt.subplots_adjust(wspace=0.4)
    if not plt_name == '':
        plt.savefig(f"trials/v_{version}/scatter_images/{plt_name}.png")
    plt.show()


def Silhouette_plot(model, data):
    visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
    visualizer.fit(data)
    return visualizer


def plot_bars_box(df, y_col, plt_name):
    col_index_name = [(df.columns.get_loc(c), c) for c in df]

    plt.figure(figsize=(8, 5))
    plt.rcParams["xtick.labelsize"] = 8
    plt.rcParams["ytick.labelsize"] = 8

    plt.subplot(1, 2, 1)
    # plt.subplot(rows, 2, i+x)
    sns.boxplot(
        data=df,
        x="cluster",
        y=y_col,
        palette="Set2",
        order=df.groupby(["cluster"])[y_col]
        .median()
        .sort_values(ascending=False)
        .index,
    )
    sns.despine()
    plt.title(
        f"BOXPLOT - CLUSTERS VS. {y_col}",
        fontsize=9,
        color="dimgrey",
        fontweight="bold",
    )
    plt.xlabel("CLUSTERS", color="dimgrey", labelpad=5, fontweight="bold", fontsize=7)
    plt.ylabel(y_col, color="dimgrey", fontweight="bold", fontsize=7)

    plt.subplot(1, 2, 2)
    # plt.subplot(rows, 2, i+x+1)
    cores = ["#66c2a5", "#8da0cb", "#fc8d62"]
    sns.set_palette(sns.color_palette(cores))
    mean = np.mean(df[y_col])
    ax = sns.barplot(
        x="cluster",
        y=y_col,
        data=df,
        order=df.groupby(["cluster"])[y_col].mean().sort_values(ascending=False).index,
    )
    sns.despine()
    ax.axhline(mean, color="r", linestyle="--")
    plt.title(f"CLUSTERS VS. {y_col}", fontsize=9, color="dimgrey", fontweight="bold")
    plt.xlabel("CLUSTERS", color="dimgrey", labelpad=5, fontweight="bold", fontsize=7)
    plt.ylabel(f"AVERAGE {y_col}", color="dimgrey", fontweight="bold", fontsize=7)

    # x += 1
    plt.subplots_adjust(wspace=0.4)
    plt.savefig(f"trials/v_{version}/results/plotBars_{y_col}.png")
    plt.show()


# OTHER GENERAL FUNCTIONS
def variance_table(data, explained_var, title):
    # Table of the amount of variation explained
    num = range(1, len(data.columns) + 1)
    df_com = pd.DataFrame(num, columns=["Number of Components"])
    ev = explained_var
    df_ev = pd.DataFrame(ev, columns=[title])
    df_ev = pd.concat([df_com, df_ev], axis=1)
    df_ev.index = df_ev.index + 1
    return df_ev


def apply_kmeans(n_clust, data_to_fit, data_to_show):
    # Apply the K-Means algorithm to 3 Clusters
    kmeans = KMeans(n_clusters=n_clust, random_state=18)
    kmeans.fit(data_to_fit)
    df = data_to_show.copy()
    df["cluster"] = kmeans.labels_

    return df, kmeans


# Evaluating metrics for several different cluster values
def metrics_table(n_cluster_range, data, model_touse, metrics):
    # Metrics
    n_clusters = []
    silhouette = []
    calinski_harabasz = []
    davies_bouldin = []
    inertia = []

    for n_cluster in n_cluster_range:
        if model_touse == "kmeans":
            model = KMeans(n_clusters=n_cluster, random_state=18)
        elif model_touse == "hc":
            model = AgglomerativeClustering(n_clusters=n_cluster)

        pred = model.fit_predict(data)

        if "silhouette" in metrics:
            silhouette.append(silhouette_score(data, pred))
        else:
            silhouette = [0] * len(n_cluster_range)
        if "calinski_harabasz" in metrics:
            calinski_harabasz.append(calinski_harabasz_score(data, pred))
        else:
            calinski_harabasz = [0] * len(n_cluster_range)
        if "davies_bouldin" in metrics:
            davies_bouldin.append(davies_bouldin_score(data, pred))
        else:
            davies_bouldin = [0] * len(n_cluster_range)
        if "inertia" in metrics:
            inertia.append(model.inertia_)
        else:
            inertia = [0] * len(n_cluster_range)

        n_clusters.append(n_cluster)

    # Results
    result = pd.DataFrame(
        {
            "Model": model_touse,
            "Clusters": n_clusters,
            "Silhouette": silhouette,
            "Calinski Harabasz": calinski_harabasz,
            "Davies Bouldin": davies_bouldin,
            "Inertia": inertia,
        }
    )

    if "silhouette" in metrics:
        result.sort_values("Silhouette", ascending=False)

    return result


def columnsnames_to_index(col_index_name, xycol_names):
    col_index_nameunzip = list(zip(*col_index_name))
    xycol = [
        (col_index_nameunzip[1].index(item[0]), col_index_nameunzip[1].index(item[1]))
        for item in xycol_names
    ]
    return xycol


def get_outliers(df, quantile_num=0.99, upper_lower='>'):
    """_summary_

    Args:
        df (_type_, optional): _description_. Defaults to aggreg_df.
        quantile_num (float, optional): _description_. Defaults to .98.

    Returns:
        dictionary: of lists, key is column name, list is value at the quartile and how many values are above the quartile
    """

    # show outliers: show how many items /rows above upper quantiles
    df_col = df.select_dtypes(include=[np.float64, np.int64]).columns.tolist()
    if upper_lower == '>':
        quantiles_dict = dict(
            [
                (
                    item,
                    [
                        df[item].quantile(quantile_num),
                        df.loc[df[item] > df[item].quantile(quantile_num)].shape[0],
                    ],
                )
                for item in df_col
            ]
        )
    else:
        quantiles_dict = dict(
            [
                (
                    item,
                    [
                        df[item].quantile(quantile_num),
                        df.loc[df[item] < df[item].quantile(quantile_num)].shape[0],
                    ],
                )
                for item in df_col
            ]
        )

    return pd.DataFrame.from_dict(quantiles_dict).rename(
        index={0: "quntile_value", 1: "rows_in_quantile"}
    )


In [ ]:
version = 10

# read in final aggregated data
df_original = pd.read_pickle("mqi_data_aggregated/aggreg_df.pkl")

# setup path to save images
paths_to_make = ["data_images", "results", "scatter_images"]
for i in range(len(paths_to_make)):
    path = pathlib.Path(f"./trials/v_{version}/{paths_to_make[i]}")
    path.mkdir(parents=True, exist_ok=True)

path = pathlib.Path(f"./trials/v_{version}/")


Fix all Null Values and apply general data clean up

In [ ]:
df_original.isna().sum()


In [ ]:
df = df_original.copy()

print(df.shape)

#Remove any models that have null values for actual vs expected, upper, lower percent difference, these models have never predicted (ie, sensor issues or something like that)
df = df.loc[~((df["actual_vs_expected_percdiff"].isna()) | (df["upper_vs_actual_percdiff"].isna()) | (df["lower_vs_actual_percdiff"].isna()))]
print(df.shape)

#remove any models that model is not at least 4 weeks old and remove models older than 3.6 years
df = df.loc[df['model_age'] > 12]

print(df.shape)

#fill any null model score with 0, fixed limit, moving average, forcast, rate of change do not have model score
df['model_score_avg'].fillna(0, inplace=True)

df.isna().sum()

In [ ]:
#convert any non numeric columns to numeric
print(df.shape)
df = pd.get_dummies(df, columns=["modeltype", "site", "position_class_group"])
print(df.shape)

Drop columns we do not want to cluster on

In [ ]:
#make Copy of data
dfcpy = df.copy()
df_clean_original = df.copy()

# Touch_freq is a sum of all these columns
action_col = [
    "actiontype_Clear Alert Status_freq",
    "actiontype_Diagnose Cleared_freq",
    "actiontype_Diagnose Set_freq",
    "actiontype_Ignore Expiration_freq",
    "actiontype_Ignore Set_freq",
    "actiontype_Model Maintenance Cleared_freq",
    "actiontype_Model Maintenance Set_freq",
    "actiontype_Note Added_freq",
    "actiontype_Quick Watch Set_freq",
    "actiontype_Stop Ignoring_freq",
    "actiontype_Watch Cleared_freq",
    "actiontype_Watch Expiration_freq",
    "actiontype_Watch Override_freq",
    "actiontype_Watch Set_freq",
]

#Equipment Model is for is not something we want to cluster on, it will be good for cluster interpretation 
position_col = [
    "position_class_group_EP",
    "position_class_group_FP-COMP_VALVE_PIPE_OTHER",
    "position_class_group_FP-EXCH",
    "position_class_group_FP-FIRED",
    "position_class_group_FP-tank-vessel",
    "position_class_group_IP-LOOPS_XMTR_OTHER",
    "position_class_group_IP-VALVE",
    "position_class_group_MP-COMP",
    "position_class_group_MP-FAN_TURBN_GRBX_OTHER",
    "position_class_group_MP-PUMP",
    "position_class_group_OP_P",
    'position_class_group_unknown'
]

#Type of Model is not something we want to cluster on, it will be good for cluster interpretation 
model_types = [
    "modeltype_APR",
    "modeltype_Fixed Limit",
    # "modeltype_Forecast",
    "modeltype_Moving Average",
    "modeltype_Rate of Change",
    "modeltype_Rolling Average",
]

#Site of Model is not something we want to cluster on, it will be good for cluster interpretation 
model_site = [
"site_FHR-CC", 
"site_FHR-PB", 
"site_Pipelines and Terminals", 
'site_unknown'
]

alert_col = [
    "HHA_freq",
    "FRQ_freq",
    "AAF_freq",
    "LLA_freq",
    "WOR_freq",
    "AAS_freq",
    "OSC_freq",
]

#No_alert_freq summarize these columns (opposite but should give same result)
dfcpy.drop(columns=alert_col, inplace=True)

#case count and frequency is not a good metric, it is not tied directly to a model but instead tied to a piece of equipment a model is on (may be misleading)
dfcpy.drop(columns=['case_count_freq', 'case_count'], inplace=True)

#Drop Columns:
dfcpy.drop(columns=model_site+action_col+position_col+model_types, inplace=True)

#drop is not something we want to cluster on, it will be good for cluster interpretation
# dfcpy.drop(columns=['currently_active_False', 'currently_active_True'], inplace=True)
#dropping currently_active_False bc more values will be 1 in currently_active_True
dfcpy.drop(columns=['currently_active_False'], inplace=True)
dfcpy.drop(columns=['currently_active_True'], inplace=True)

#drop model_score_avg because not all models have a model score ie. Fixedlimit and rolling average
dfcpy.drop(columns=['model_score_avg'], inplace=True)


# #action_note_len_avg may lead to incorrect clustering, there are alot of auto generated notes etc, that has collinearity with touch_freq
# dfcpy.drop(columns=['action_note_len_avg'], inplace=True)

#drop due to collinearity with model_build_freq
dfcpy.drop(columns=['model_save_freq'], inplace=True)
# dfcpy.drop(columns=['model_build_freq'], inplace=True)

dfcpy.drop(columns=['noAlert_freq'], inplace=True)
# dfcpy.drop(columns=['in_alert_freq'], inplace=True)

#drop due to collinearity with actual_vs_expected_percdiff. Using upper vs expected, because fixed limit models do not difference in actual vs expected (ie. == 0)
#drop actual_vs_expected_percdiff because not all models have a model score ie. Fixedlimit and rolling average
dfcpy.drop(columns=['actual_vs_expected_percdiff'], inplace=True)
# dfcpy.drop(columns=['actual_vs_expected_percdiff', 'lower_vs_actual_percdiff'], inplace=True)

dfcpy.columns

In [ ]:
#look correlations at data distributions
plt_heat(dfcpy, fig_hight=10, fig_width=10, fntsze=10)
plt_distplot(dfcpy, plt_name='raw', fig_hight=35, fig_width=15)
# plt_boxplot(dfcpy, plt_name='raw', fig_hight=35, fig_width=15)


In [ ]:
# #drop model age due to high collinearity  with currently active true and model build_freq
dfcpy.drop(columns=['model_age'], inplace=True)


Drop Outliers

In [ ]:
#upper outliers
outliers_high = get_outliers(dfcpy, 0.98, '>')
print(dfcpy.shape)
for col, val in outliers_high.T.iterrows():
    dfcpy = dfcpy.loc[dfcpy[col] <= val.values[0]]
print(dfcpy.shape)
dfcpy.describe().T

#lower outliers only for model_build_freq
outliers_high = get_outliers(dfcpy[['model_build_freq']], 0.002, '<')
print(dfcpy.shape)
for col, val in outliers_high.T.iterrows():
    dfcpy = dfcpy.loc[dfcpy[col] >= val.values[0]]
print(dfcpy.shape)
dfcpy.describe().T



# model=IsolationForest(n_estimators=150, max_samples='auto', contamination=float(0.1), max_features=1.0)
# model.fit(dfcpy)

# IsolationForest(contamination=0.1, n_estimators=150)

# scores=model.decision_function(dfcpy)
# anomaly=model.predict(dfcpy)

# dfcpy['scores']=scores
# dfcpy['anomaly']=anomaly

# anomaly = dfcpy.loc[dfcpy['anomaly']==-1]
# anomaly_index = list(anomaly.index)
# print('Total number of outliers is:', len(anomaly))

# # dropping outliers
# dfcpy = dfcpy.drop(anomaly_index, axis = 0)
# dfcpy.drop(columns=['scores', 'anomaly'], inplace=True)
# print(dfcpy.shape)

there are alot of models with zero touches and zero alerts, this is causing data to have a large skew. will need to remove some of these to try and get better distribution


In [ ]:
fig = sns.pairplot(dfcpy[0:])
# fig.savefig(f"trials/v_{version}/scatter_images/pairplot_kmeans_3.png")

In [ ]:
dfcpy.columns

In [ ]:
# # random sample to remove some zero values
# print(dfcpy.shape)
# touches_random_sample = dfcpy.loc[((dfcpy['touches_count_freq']==0))]
# index_to_remove = touches_random_sample.sample(int(touches_random_sample.shape[0]//2)).index
# dfcpy.drop(index=index_to_remove, inplace=True)
# print(dfcpy.shape)

# print(dfcpy.shape)
# touches_random_sample = dfcpy.loc[((dfcpy['in_alert_freq']==0))]
# index_to_remove = touches_random_sample.sample(int(touches_random_sample.shape[0]//2)).index
# dfcpy.drop(index=index_to_remove, inplace=True)
# print(dfcpy.shape)

# print(dfcpy.shape)
# touches_random_sample = dfcpy.loc[((dfcpy['upper_vs_actual_percdiff']==0))]
# index_to_remove = touches_random_sample.sample(int(touches_random_sample.shape[0]//2)).index
# dfcpy.drop(index=index_to_remove, inplace=True)
# print(dfcpy.shape)

# print(dfcpy.shape)
# touches_random_sample = dfcpy.loc[((dfcpy['lower_vs_actual_percdiff']==0))]
# index_to_remove = touches_random_sample.sample(int(touches_random_sample.shape[0]//2)).index
# dfcpy.drop(index=index_to_remove, inplace=True)
# print(dfcpy.shape)


In [ ]:
#update original data frame with removed rows (need this to match clusters later)
df_clean_original = df_clean_original.loc[df_clean_original.index.isin(dfcpy.index)]
df_clean_original.shape

In [ ]:
# dfcpy.skew().sort_values(ascending=False)

Apply log transform to help distribution

In [ ]:
columns_to_log = [
    col for col in dfcpy.columns if len(dfcpy[col].value_counts()) != 2
]
dfcpy[columns_to_log] = dfcpy[columns_to_log].apply(lambda x: np.log(x+1))

plt_boxplot(dfcpy, plt_name='raw', fig_hight=10, fig_width=20, fntsze=10)
plt_distplot(dfcpy, plt_name='raw', fig_hight=20, fig_width=30, fntsze=10)

NORMALIZE DATA

In [ ]:
# normalize with Min-max
df_norm = dfcpy.copy()
# get only columns that are not 0 or 1 (ie have range of values)
columns_to_norm = [
    col for col in df_norm.columns if len(df_norm[col].value_counts()) != 2
]
min_max_scaler = preprocessing.MinMaxScaler()
df_norm[columns_to_norm] = min_max_scaler.fit_transform(df_norm[columns_to_norm])


# # standardize with mean
# df_stand = dfcpy.copy()
# # get only columns that are not 0 or 1 (ie have range of values)
# columns_to_stand = [
#     col for col in df_stand.columns if len(df_stand[col].value_counts()) != 2
# ]
# stand_scaler = preprocessing.StandardScaler()
# df_stand[columns_to_stand] = stand_scaler.fit_transform(df_stand[columns_to_stand])

In [ ]:
df_norm.describe().round(2).T

In [ ]:
plt_boxplot(df_norm, plt_name='raw', fig_hight=15, fig_width=20, fntsze=10)
plt_distplot(df_norm, plt_name='raw', fig_hight=15, fig_width=20, fntsze=10)
# plt_histo(df_norm, plt_name='raw', fig_hight=15, fig_width=20, fntsze=10, bns=150)

In [ ]:
features_df = df_norm.copy()
# features_df = df_stand.copy()

START PCA: Determine how many features to use for PCA

In [ ]:
# ***********Start PCA *******************
use_PCA = False

if use_PCA:
    pca = PCA()
    df_pca = pca.fit_transform(features_df)
    var1 = variance_table(features_df, pca.explained_variance_, "Explained Variance")
    var2 = variance_table(
        features_df,
        (pca.explained_variance_ratio_ * 100),
        "Percentage of Explained Variance",
    )
    var3 = variance_table(
        features_df,
        (pca.explained_variance_ratio_.cumsum() * 100),
        "Cumulative Percentage of Explained Variance",
    )
else:
    data = features_df

In [ ]:
if use_PCA:
    print(var1.head(5))
    print(var2.head(5))
    print(var3.head(10))


In [ ]:
if use_PCA:
    variance_bars_plt()
    percent_variance_plt_ratio(
        [pca.explained_variance_ratio_, pca.explained_variance_ratio_.cumsum()],
        ["Percentage of Explained Variance", "Cumulative Percentage of Explained Variance"],
        ["Components", "Components"],
        [0, 0.8],
    )


Generally it is recommended to use at least 80% of the variance when applying PCA.

In [ ]:
if use_PCA:
    # with more than 80% of the variance
    pca1 = PCA(n_components=4, random_state=18)
    df_pca1 = pca1.fit_transform(features_df)
    df_pca1.shape

START: Determine how many Cluster to use

Silhouette: closer to 1 the better
    shows how similar each sample is to other points in its own cluster relative to other clusters. Positive values == closer to point in own cluster
    0 values indicate point is on the border
    neg values indicate samples are not clustered well and point should be in another cluster

Calinski Harabasz: the higher the value the better the clustering:
    measures the relationship between variations within its own cluster and variations between clusters 
    lower values can indicate the clusters are overlapping

Davies Bouldin: the smaller this value the better:
    is a measurement of similarity between data samples in a cluster. The value measures the similarity between each cluster and its closet neighbors
    the smaller this value shows that an item in a cluster is less like another cluster, ie. good clustering


In [ ]:

# Plot metrics to find number of cluster
plt_kelbow_metric(
    data,
    KMeans(random_state=18),
    (2, 9),
    ["distortion", "silhouette", "calinski_harabasz"],
    "kmeans",
)


In [ ]:
clst_scores_df = metrics_table(
    range(2, 8),
    data,
    "kmeans",
    ["inertia", "silhouette", "calinski_harabasz", "davies_bouldin"],
)
clst_scores_df


Silhouette: closer to 1 the better

Inertia: the lower this value the better

Calinski Harabasz: the higher the value the better the clustering:

Davies Bouldin: the smaller this value the better:

Silhouette analysis can be used visually see how each cluster relates to another. 
values on the x-axis closer +1 are far away from neighboring cluster
values closer to zero are on or near the boarder line between neighboring clusters
negative values  indicate it may be in the wrong cluster

The dotted line represents average Silhouette score

In [ ]:

n_clust_try = [2, 3, 4]
for i in range(len(n_clust_try)):
    vis = Silhouette_plot(
        KMeans(n_clusters=n_clust_try[i], random_state=18), data
    ).show()
    vis.get_figure().savefig(
        f"trials/v_{version}/scatter_images/silhouettvis_cluster-{n_clust_try[i]}.png"
    )


In [ ]:
features_df.columns


In [ ]:
# get column index / name for plot
col_index_name = [(features_df.columns.get_loc(c), c) for c in features_df]

# selecting which columns to plot vs each other.
# xycol_names = [('actual_vs_expected_percdiff', 'touches_count_freq'),
#                     ('model_build_freq', 'touches_count_freq'),
#                     ('model_build_freq', 'noAlert_freq'),
#                     ('noAlert_freq', 'touches_count_freq'),
#                     ]

# xycol_names = [
#     ("actual_vs_expected_percdiff", "actiontype_Quick Watch Set_freq"),
#     ("upper_vs_actual_percdiff", "model_save_freq"),
#     ("FRQ_freq", "model_save_freq"),
#     ("actiontype_Note Added_freq", "model_build_freq"),
# ]


xycol_names = [
    ("upper_vs_actual_percdiff", "in_alert_freq"),
    ("upper_vs_actual_percdiff", "touches_count_freq"),
    ("upper_vs_actual_percdiff", "in_alert_freq"),
    ("touches_count_freq", "model_build_freq"),
    ("touches_count_freq", "in_alert_freq"),
    ("in_alert_freq", "model_build_freq"),
]


# convert column name tuples to indexes

xycol = columnsnames_to_index(col_index_name, xycol_names)

# calculate Kmeans with PCA or original data, then plot with original data

n_clust = 2
df2, kmeans2 = apply_kmeans(n_clust, data, features_df)
plot_general(df2, xycol, f"clust_kmeans-{n_clust}", figsiz=(30, 40), fntsze=14)

n_clust = 3
df3, kmeans3 = apply_kmeans(n_clust, data, features_df)
plot_general(df3, xycol, f"clust_kmeans-{n_clust}", figsiz=(30, 40), fntsze=14)

n_clust = 4
df4, kmeans4 = apply_kmeans(n_clust, data, features_df)
plot_general(df4, xycol, f"clust_kmeans-{n_clust}", figsiz=(30, 40), fntsze=14)



In [ ]:
fig = sns.pairplot(df3[0:],hue='cluster')
fig.savefig(f"trials/v_{version}/scatter_images/pairplot_kmeans_3.png")


In [ ]:
fig = px.scatter_3d(df3,x="upper_vs_actual_percdiff",y="touches_count_freq",z="in_alert_freq",
                    color='cluster',
                    title=f'KMeans 3')
fig.show()

In [ ]:
if use_PCA:
    # Apply PCA  components for preview
    n_comp_2 = 2
    pca2 = PCA(n_components=n_comp_2, random_state=18)
    df_pca2 = pca2.fit_transform(features_df)

    n_comp_3 = 3
    pca3 = PCA(n_components=n_comp_3, random_state=18)
    df_pca3 = pca3.fit_transform(features_df)


    df_pca2_kmn = pd.DataFrame(data=df_pca2, columns=[f"PCA{i+1}" for i in range(n_comp_2)])
    df_pca3_kmn = pd.DataFrame(data=df_pca3, columns=[f"PCA{i+1}" for i in range(n_comp_3)])

    n_clust = 3
    df_kmeans_pca2, kmeans_pca2 = apply_kmeans(n_clust, df_pca2, df_pca2_kmn)
    df_kmeans_pca3, kmeans_pca3 = apply_kmeans(n_clust, df_pca3, df_pca3_kmn)


In [ ]:
if use_PCA:
    plot_general(df_kmeans_pca2, [(0, 1)], f"kmeans_PCA_clust-{n_comp_2}")

In [ ]:
if use_PCA:
    fig = px.scatter_3d(df_kmeans_pca3,x="PCA1",y="PCA2",z="PCA3",
                        color='cluster',
                        title=f'KMeans {n_comp_3} ')
    fig.show()


In [ ]:
# Final K-Means model
final_k_clusters = 3

final_df_kmn = metrics_table(
    range(final_k_clusters, final_k_clusters + 1),
    data,
    "kmeans",
    ["inertia", "silhouette", "calinski_harabasz", "davies_bouldin"],
)

final_df_kmn


In [ ]:
# **********START DBSCAN METHOD*******************
# NearestNeighbors

# Final K-Means model
nbrs = NearestNeighbors(n_neighbors=3).fit(data)

# Calculating the distances and indices of the k nearest neighbors
distances, indices = nbrs.kneighbors(data)

# Plot
plt.plot(sorted(distances[:, 1]), "r-")
plt.title(
    "DISTANCES BETWEEN THE K NEAREST NEIGHBORS OF EACH POINT",
    fontsize=11,
    color="dimgrey",
    fontweight="bold",
    y=1,
)
plt.xlabel(
    "INDICES  K NEAREST NEIGHBORS",
    color="dimgrey",
    labelpad=5,
    fontweight="bold",
    fontsize=8,
)
plt.ylabel(
    "DISTANCE TO NEAREST NEIGHBOR", color="dimgrey", fontweight="bold", fontsize=8
)


plt.show()


In [ ]:
# eps_lst = np.linspace(0.1, 0.2, 6)
# eps_lst = [.08, .1, .12, .13, .15, .18, .2, .23, .25, .275]
# eps_lst = [.005, .01, .02, .04, .05, .06, .08, .1]
# eps_lst = [.25, .3, .3125, .325, .35, .375, .4, .5, .55]
eps_lst = [.03, .04, .05, .06, .07, .1, .13, .15]
min_sample = [2, 3, 5, 10, 20, 40, 50, 60, 70]
print(eps_lst, min_sample)


In [ ]:
# Final K-Means model
# Defines the parameter search space
param_grid = {"eps": eps_lst, "min_samples": min_sample}

# Create the DBSCAN object
dbscan = DBSCAN()

# Creates the GridSearchCV object
grid_search = GridSearchCV(dbscan, param_grid, scoring=silhouette_score, n_jobs=-1)

# Run the grid search
grid_search.fit(data)

# Displays the best parameters found
print("Best parameters found: ", grid_search.best_params_)


In [ ]:
# Instance of DBSCAN
dbscan = DBSCAN(eps=0.03, min_samples=10, n_jobs=-1).fit(data)
# Getting the cluster labels
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)


In [ ]:
np.linspace(0.02, 0.1, 20)
# np.linspace(0.1, 0.3, 20)

In [ ]:
eps = []
min_samples = []
silhouette = []
calinski_harabasz = []
davies_bouldin = []
n_clusters = []
metrics_dbscan = []

for eps in np.linspace(0.02, 0.12, 20):
    for min_sample in range(2, 60, 5):
        dbscan = DBSCAN(eps=eps, min_samples=min_sample, n_jobs=-1)
        dbscan.fit(data)
        pred = dbscan.labels_

        if len(np.unique(pred)) > 1:
            silhouette = silhouette_score(data, pred)
            calinski_harabasz = calinski_harabasz_score(data, pred)
            davies_bouldin = davies_bouldin_score(data, pred)
            metrics_dbscan.append(
                (
                    eps,
                    min_sample,
                    silhouette,
                    calinski_harabasz,
                    davies_bouldin,
                    len(set(dbscan.labels_)),
                )
            )

df_dbscan = pd.DataFrame(
    metrics_dbscan,
    columns=[
        "Eps",
        "Min Samples",
        "Silhouette",
        "Calinski Harabasz",
        "Davies Bouldin",
        "Number of Clusters",
    ],
)




In [ ]:
df_dbscan.sort_values("Silhouette", ascending=False).head(100)

Pick EPS 0.173684, Min Samples 47

In [ ]:
silhouette_coefs = []

min_sample_points = [5, 10, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 42, 60, 70]
for min_points in min_sample_points:
    db = DBSCAN(eps=0.100, min_samples=min_points, n_jobs=-1)
    db.fit(data)
    silhouette_coefs.append(silhouette_score(data, db.labels_))

plt.figure(figsize=(6, 3), dpi=120)
plt.plot(
    min_sample_points, silhouette_coefs, color="r", marker="."
)
plt.title(
    "SILHOUETTE SCORE - MINIMUM CLUSTERS SAMPLES",
    fontsize=9,
    color="dimgrey",
    fontweight="bold",
    y=1.02,
)
plt.xlabel(
    "MINIMUM SAMPLES OF EACH CLUSTERS",
    color="dimgrey",
    labelpad=5,
    fontweight="bold",
    fontsize=6,
)
plt.ylabel("SILHOUETTE SCORE", color="dimgrey", fontweight="bold", fontsize=6)
plt.show()


In [ ]:
# DBSCAN instance
dbscan = DBSCAN(eps=0.045678, min_samples=30, n_jobs=-1).fit(data)

# Getting the cluster labels
dbscan_labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present
n_clusters_ = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise_ = list(dbscan_labels).count(-1)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

df_bscan = features_df.copy()
# Append DBSCAN clusters result to Dataframe
df_bscan["cluster"] = dbscan_labels

# Quantity of each cluster
df_bscan["cluster"].value_counts()

In [ ]:
plot_general(df_bscan, xycol, f"clust_dbscan{n_clusters_}")

In [ ]:
fig = sns.pairplot(df_bscan[0:],hue='cluster')
fig.savefig(f"trials/v_{version}/scatter_images/pairplot_DB.png")

In [ ]:
if use_PCA:
    # Plot DBscan clusters with the 2 PCAs
    df_pca2_db = df_pca2.copy()
    df_pca3_db = df_pca3.copy()

    # Dataframe for two components
    df_pca2_db = pd.DataFrame(data=df_pca2, columns=[f"PCA{i+1}" for i in range(n_comp_2)])
    df_pca3_db = pd.DataFrame(data=df_pca3, columns=[f"PCA{i+1}" for i in range(n_comp_3)])

    # Append cluster labels to the Dataframe
    df_pca2_db = pd.concat([df_pca2_db, pd.DataFrame({"cluster": dbscan_labels})], axis=1)
    df_pca3_db = pd.concat([df_pca3_db, pd.DataFrame({"cluster": dbscan_labels})], axis=1)


In [ ]:
if use_PCA:
    plot_general(df_pca2_db, [(0, 1)], f"dbscan_PCA")

In [ ]:
if use_PCA:
    fig = px.scatter_3d(df_pca3_db,x="PCA1",y="PCA2",z="PCA3",
                        color='cluster',
                        title=f'KMeans {n_clust} ')
    fig.show()

In [ ]:
# Final DBSCAN model
# Metrics
metrics = []

dbscan = DBSCAN(eps=0.18, min_samples=25).fit(data)
pred = dbscan.labels_

model = "DBSCAN"
n_clusters = 3
silhouette = silhouette_score(data, pred)
calinski_harabasz = calinski_harabasz_score(data, pred)
davies_bouldin = davies_bouldin_score(data, pred)

metrics.append((model, n_clusters, silhouette, calinski_harabasz, davies_bouldin))

final_df_db = pd.DataFrame(
    metrics,
    columns=[
        "Model",
        "Clusters",
        "Silhouette",
        "Calinski Harabasz",
        "Davies Bouldin",
    ],
)

final_df_db


In [ ]:
# **********START Hierarchical Clustering METHOD*******************
dend = linkage(data, "ward")
plt.axhline(y=16, color="black", linestyle="--")
dendrogram(dend)

plt.savefig(f"trials/v_{version}/scatter_images/Dendo.png")
plt.show()


In [ ]:
plt_kelbow_metric(
    data,
    "",
    "",
    ["distortion", "silhouette", "calinski_harabasz"],
    "AgglomerativeClustering",
)


In [ ]:
hc_scores_df = metrics_table(
    range(2, 8), data, "hc", ["silhouette", "calinski_harabasz", "davies_bouldin"]
)
hc_scores_df


In [ ]:
nclusters = 3
hc = AgglomerativeClustering(n_clusters=nclusters)

# Fit the model to the data
hc.fit(data)

# Prints the labels for each example
hc_labels = hc.labels_

df_hc = features_df.copy()
df_hc["cluster"] = hc_labels

plot_general(df_hc, xycol, f"Clust_HC{nclusters}", (20, 30), 10)


In [ ]:
fig = sns.pairplot(df_hc[0:],hue='cluster')
fig.savefig(f"trials/v_{version}/scatter_images/pairplot_HC.png")

In [ ]:

fig = px.scatter_3d(df_hc,x="actual_vs_expected_percdiff",y="touches_count_freq",z="noAlert_freq",
                    color='cluster',
                    title=f'KMeans 3 ')
fig.show()

In [ ]:
if use_PCA:
    # Plot DBscan clusters with the 2 PCAs
    df_pca2_hc = df_pca2.copy()
    df_pca3_hc = df_pca3.copy()

    # Dataframe for two components
    df_pca2_hc = pd.DataFrame(data=df_pca2, columns=[f"PCA{i+1}" for i in range(n_comp_2)])
    df_pca3_hc = pd.DataFrame(data=df_pca3, columns=[f"PCA{i+1}" for i in range(n_comp_3)])

    # Append cluster labels to the Dataframe
    df_pca2_hc = pd.concat([df_pca2_hc, pd.DataFrame({"cluster": hc_labels})], axis=1)
    df_pca3_hc = pd.concat([df_pca3_hc, pd.DataFrame({"cluster": hc_labels})], axis=1)


In [ ]:
if use_PCA:
    plot_general(df_pca2_hc, [(0, 1)], f"dbscan_HC_PCA")


In [ ]:
if use_PCA:    
    fig = px.scatter_3d(df_pca3_hc,x="PCA1",y="PCA2",z="PCA3",
                        color='cluster',
                        title=f'KMeans {n_clust} ')
    fig.show()

In [ ]:
# Final Hierarchical Clustering Model
final_hc_clusters = 3

final_df_hc = metrics_table(
    range(final_hc_clusters, final_hc_clusters + 1),
    data,
    "hc",
    ["silhouette", "calinski_harabasz", "davies_bouldin"],
)

final_df_hc


In [ ]:
# **********Final Results**************
# Comparison table between algorithms
# df_comp = pd.concat([final_df_kmn, final_df_db, final_df_hc])
df_comp = pd.concat([final_df_kmn, final_df_hc])
df_comp.sort_values("Silhouette", ascending=False)


In [ ]:
# final_lables = dbscan.labels_
# final_lables = kmeans4.labels_
final_lables = kmeans3.labels_
# final_lables = kmeans2.labels_

# final_lables = hc.labels_

In [ ]:
df_clean_original.columns

In [ ]:
col_to_use = [
    "currently_active_False",
    "currently_active_True",
    "model_save_freq",
    "model_build_freq",
    "model_score_avg",
    "noAlert_freq",
    "model_age",
    "actual_vs_expected_percdiff",
    "upper_vs_actual_percdiff",
    "lower_vs_actual_percdiff",
    'case_count',
    "case_count_freq",
    "touches_count_freq",
    "modeltype_APR",
    "modeltype_Fixed Limit",
    "modeltype_Rolling Average",
    "site_FHR-CC",
    "site_FHR-PB",
    "site_Pipelines and Terminals",
    "site_unknown",
    "position_class_group_EP",
    "position_class_group_FP-COMP_VALVE_PIPE_OTHER",
    "position_class_group_FP-EXCH",
    "position_class_group_FP-FIRED",
    "position_class_group_FP-tank-vessel",
    "position_class_group_IP-LOOPS_XMTR_OTHER",
    "position_class_group_IP-VALVE",
    "position_class_group_MP-COMP",
    "position_class_group_MP-FAN_TURBN_GRBX_OTHER",
    "position_class_group_MP-PUMP",
    "position_class_group_OP_P",
    'position_class_group_unknown'
]

df_clean_original = df_clean_original[col_to_use].copy()

# Append the final result of the clusters to the original Dataframe
df_clean_original['cluster'] = final_lables


case_count_df = df_clean_original.groupby("cluster", group_keys=False)[
        ["case_count"]
    ].sum()
case_count_df.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(5, 4))

plt.rcParams["xtick.labelsize"] = 8
plt.rcParams["ytick.labelsize"] = 8

total = float(df_clean_original.shape[0])
cores = ["#8da0cb", "#fc8d62", "#66c2a5"]
sns.set_palette(sns.color_palette(cores))
ax = sns.countplot(
    x="cluster",
    data=df_clean_original,
    order=df_clean_original["cluster"].value_counts().index,
)
sns.despine()
plt.title("CLUSTERS DISTRIBUTION", fontsize=9, color="dimgrey", fontweight="bold")
for p in ax.patches:
    percentage = "{:.1f}%".format(100 * p.get_height() / total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha="center")
plt.xlabel("CLUSTERS", color="dimgrey", labelpad=5, fontweight="bold", fontsize=7)
plt.ylabel("COUNT", color="dimgrey", fontweight="bold", fontsize=7)
plt.savefig(f"trials/v_{version}/results/bar-cluster_distribution.png")
plt.show()


In [ ]:
# Plotting columns by clusters
plt.figure(figsize=(20, 120))
x = 1
for d in df_clean_original:
    if not d == "cluster":
        plt.subplot(((df_clean_original.shape[1] // 2) + 1), 2, x)
        sns.kdeplot(data=df_clean_original, x=d, hue="cluster", palette="Set2")
        x += 1

plt.savefig(f"trials/v_{version}/results/kdeplot.png")


In [ ]:
for c in df_clean_original:
    if not c == "cluster":
        grid = sns.FacetGrid(df_clean_original, col="cluster").map(plt.hist, c)
        plt.savefig(f"trials/v_{version}/results/hist_{c}.png")


In [ ]:
# Plot of clusters by columns
plt.figure(figsize=(20, 100))
x = 1
for d in df_clean_original:
    if not d == "cluster":
        plt.subplot(((df_clean_original.shape[1] // 2) + 1), 3, x)
        sns.stripplot(data=df_clean_original, x="cluster", y=d, palette="Set2")
        x += 1

plt.savefig(f"trials/v_{version}/results/stripplot.png")


In [ ]:
col_to_plt = df_clean_original.columns.tolist()

for i in range(len(col_to_plt)):
    if not col_to_plt[i] == "cluster":
        plot_bars_box(df_clean_original, col_to_plt[i], f"kmeans_clust-{n_clust}_{i}")

plot_bars_box(case_count_df, 'case_count', f"kmeans_clust-{n_clust}_caseCount")

In [ ]:
# plt.figure()
# sns.jointplot(x=df_clean_original[i], y=data["Spent"], hue =data["Clusters"], kind="kde", palette=pal)
# plt.show()


# plt.figure()
# sns.jointplot(x=df_clean_original["model_age"], y=df_clean_original["model_score_avg"], hue=df_clean_original["cluster"], kind="kde")



# col_to_plt = df_clean_original.columns.tolist()

# for i in range(len(col_to_plt)):
#     if not col_to_plt[i] == "cluster":
#         plt.figure()
#         sns.jointplot(x=df_clean_original[i], y=df_clean_original["model_score_avg"], hue =data["Clusters"], kind="kde")
        # plot_bars_box(df_clean_original, col_to_plt[i], f"kmeans_clust-{n_clust}_{i}")